In [1]:
import pandas as pd
import numpy as np
import json
import os

# df = pd.read_json('logs/n100000_s100_t10_everynth.json', orient ='index')
df = pd.DataFrame()


for filename in os.listdir('../logs'):
    f = os.path.join('../logs', filename)
    if os.path.isfile(f) & f.endswith(".json"):
        # print(filename)
        dft = pd.read_json(f)
        dft['datapoints'] = filename.split('_')[0][1:]
        dft['samples'] = filename.split('_')[1][1:]
        dft['traces'] = filename.split('_')[2][1:]
        dft['sampling algorithm'] = filename.split('_')[3].split('.')[0]
        df = df.append(dft, ignore_index=True)
        
df['message']= df['message'].str.split('\"').str[-2]
df.drop(df[df['level'].eq('SEVERE')].index, inplace=True)
df.drop('source', axis=1, inplace=True)
df.reset_index()
df
# warnings = df[df['level'].eq('ERROR')]
# warnings


,level,message,timestamp,datapoints,samples,traces,sampling algorithm
0,INFO,100.0%,2023-03-19 18:08:13.587,10000000,1000,10,everynth
1,DEBUG,time (visible): 2520.93701171875 ms,2023-03-19 18:08:16.893,10000000,1000,10,everynth
2,DEBUG,render time (visible): 56.18798828125 ms,2023-03-19 18:08:16.893,10000000,1000,10,everynth
3,DEBUG,time (invisible): 0.797119140625 ms,2023-03-19 18:08:16.894,10000000,1000,10,everynth
4,DEBUG,time (full): 2522.634033203125 ms,2023-03-19 18:08:16.895,10000000,1000,10,everynth
...,...,...,...,...,...,...,...
1480,DEBUG,time (visible): 2136.897705078125 ms,2023-03-19 18:33:19.775,100000,4000,50,everynth
1481,DEBUG,render time (visible): 47.073974609375 ms,2023-03-19 18:33:19.776,100000,4000,50,everynth
1482,DEBUG,time (invisible): 2320.608154296875 ms,2023-03-19 18:33:22.097,100000,4000,50,everynth
1483,DEBUG,render time (invisible): 54.5732421875 ms,2023-03-19 18:33:22.097,100000,4000,50,everynth


In [2]:
mask = df['message'].str.contains('%')
percentage = df[mask]
percentage
df.loc[mask, 'percentage'] = percentage['message']
df['percentage'].fillna(method='ffill', inplace=True)
df.drop(percentage.index, inplace=True)
df = df.reset_index()
df[['message', 'percentage','timestamp']]

,message,percentage,timestamp
0,time (visible): 2520.93701171875 ms,100.0%,2023-03-19 18:08:16.893
1,render time (visible): 56.18798828125 ms,100.0%,2023-03-19 18:08:16.893
2,time (invisible): 0.797119140625 ms,100.0%,2023-03-19 18:08:16.894
3,time (full): 2522.634033203125 ms,100.0%,2023-03-19 18:08:16.895
4,render time (invisible): None,100.0%,2023-03-19 18:08:16.896
...,...,...,...
1345,time (visible): 2136.897705078125 ms,10.0%,2023-03-19 18:33:19.775
1346,render time (visible): 47.073974609375 ms,10.0%,2023-03-19 18:33:19.776
1347,time (invisible): 2320.608154296875 ms,10.0%,2023-03-19 18:33:22.097
1348,render time (invisible): 54.5732421875 ms,10.0%,2023-03-19 18:33:22.097


In [3]:
df['type']= df['message'].str.split().str[0]
df['update'] = df['message'].str.split("(").str[1].str.split(")").str[0]
df['time (ms)'] = df['message'].str.split(":").str[1].str.split().str[0]
df['time (ms)']

0        2520.93701171875
1          56.18798828125
2          0.797119140625
3       2522.634033203125
4                    None
              ...        
1345    2136.897705078125
1346      47.073974609375
1347    2320.608154296875
1348        54.5732421875
1349     4458.48388671875
Name: time (ms), Length: 1350, dtype: object

In [4]:
df['type'] = df['type'].apply(lambda x: x + ' time' if x == 'render' else x)
df['type']

0              time
1       render time
2              time
3              time
4       render time
           ...     
1345           time
1346    render time
1347           time
1348    render time
1349           time
Name: type, Length: 1350, dtype: object

In [5]:
df_vanilla = pd.read_csv('../logs/processed_logs_vanilla.csv')
df_vanilla = df_vanilla.drop(df_vanilla.columns[0],axis=1)

df_vanilla_visible = df_vanilla[df_vanilla['update'].eq('visible') & df_vanilla['type'].eq('time')]
df_visible = df[df['update'].eq('visible') & df['type'].eq('time')]

df_visible['version'] = 'async trace update'
df_vanilla_visible['version'] = 'vanilla'

# df_visible = df_visible.append(df_vanilla_visible)
df_visible['time (ms)'] = pd.to_numeric(df_visible['time (ms)'])
df_vanilla_visible['time (ms)'] = pd.to_numeric(df_vanilla_visible['time (ms)'])


C:\Users\isisg\AppData\Local\Temp\ipykernel_30812\2800390986.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_visible['version'] = 'async trace update'
C:\Users\isisg\AppData\Local\Temp\ipykernel_30812\2800390986.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vanilla_visible['version'] = 'vanilla'
C:\Users\isisg\AppData\Local\Temp\ipykernel_30812\2800390986.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

In [6]:
from natsort import natsort_keygen

mean_df = df_visible.groupby(['datapoints', 'samples','traces','sampling algorithm', 'percentage', 'version'])['time (ms)'].mean().reset_index()
mean_vanilla_df = df_vanilla_visible.groupby(['datapoints', 'samples','traces','sampling algorithm', 'percentage', 'version'])['time (ms)'].mean().reset_index()
mean_vanilla_df = mean_vanilla_df.sort_values(by=['datapoints','samples','traces','sampling algorithm', 'percentage'],key=natsort_keygen())
mean_df = mean_df.sort_values(by=['datapoints','samples','traces','sampling algorithm', 'percentage'],key=natsort_keygen())
# mean_df = mean_df.reset_index()

In [7]:
mean_df['diff'] = mean_df['time (ms)'] - mean_vanilla_df['time (ms)']
mean_df['ratio'] = mean_df['time (ms)'] / mean_vanilla_df['time (ms)']
mean_df['time (ms) vanilla'] = mean_vanilla_df['time (ms)']
mean_df['percentage'] = pd.to_numeric(mean_df['percentage'].str.split('.').str[0])
mean_df.to_csv('../logs/processed_logs.csv')

In [8]:
import seaborn as sns

